# Binary to Decimal Conversion with Brainf*ck

The following is a program, written in brainfuck, to translate binary numbers into decimal numbers.  If you are unfamiliar with brainfuck, check out the following [tutorial](https://learnxinyminutes.com/docs/brainfuck/).  

The input is limited to numbers which have single-digit decimal representation since the output is contained in a single brainfuck cell and is interpreted as an ascii character.  If you have an ascii table and are willing to subtract 48 from something, then the program will work for inputs up to 255 decimal, or the maximum value of a brainfuck cell.

``` brainfuck
-+ [ - < + ] ->, [ + [ - < + ] ->>>[-]+ [ - < + ] ->>>>[-]+ [ - < + ] ->>[-+ [ - < + ] ->>>++ [ - < + ] ->>>>++ [ - < + ] ->>]+ [ - < + ] ->>[-]+ [ - < + ] ->>>>[-+ [ - < + ] ->>++ [ - < + ] ->>>>]+ [ - < + ] ->>>>[-]+ [ - < + ] ->>>[-+ [ - < + ] ->>++ [ - < + ] ->>>>++ [ - < + ] ->>>]+ [ - < + ] ->>>[-]+ [ - < + ] ->>>>[-+ [ - < + ] ->>>++ [ - < + ] ->>>>]+ [ - < + ] ->------------------------------------------------+ [ - < + ] ->>>[-]+ [ - < + ] ->[-+ [ - < + ] ->>++ [ - < + ] ->>>++ [ - < + ] ->]+ [ - < + ] ->[-]+ [ - < + ] ->>>[-+ [ - < + ] ->++ [ - < + ] ->>>]+ [ - < + ] ->,]+ [ - < + ] ->>+ [ - < + ] ->>++++++++++++++++++++++++++++++++++++++++++++++++.
```

The code you see above works!  If you don't believe me, you can test it in this [interpreter](https://www.nayuki.io/page/brainfuck-interpreter-javascript).

It is a well accepted best practice not to write brainfuck directly (or at all, for that matter).  Therefore, we build up a bunch of reusable strings with haskell.

![Brainfuck Best Practices](./generate.png)

In [2]:
gohome = "+ [ - < + ] -"

In [3]:
goto addr = gohome ++ replicate addr '>'

In [4]:
zeroCurrentCell = "[-]"

In [5]:
zeroCell addr = goto addr ++ zeroCurrentCell

In [6]:
addToCurrentCell val = replicate val '+'

In [21]:
setCell addr val = zeroCell addr ++ addToCurrentCell val

In [22]:
-- warning: zeros cell at addr1
destrAddTo addr1 addr2 = goto addr1 ++ "[-" ++ goto addr2 ++ "+" ++ goto addr1 ++ "]"

In [23]:
-- warning: zeros cell at addr1
destrMoveTo addr1 addr2 = zeroCell addr2 ++ destrAddTo addr1 addr2

In [24]:
addTo addr1 addr2 safeSpace = 
    zeroCell safeSpace ++
    goto addr1 ++ "[-" ++ goto addr2 ++ "+" ++ goto safeSpace ++ "+" ++ goto addr1 ++ "]" ++
    destrMoveTo safeSpace addr1 --zeros cell at safe space

In [25]:
copyTo addr1 addr2 safeSpace = zeroCell addr2 ++ addTo addr1 addr2 safeSpace

In [26]:
readDoWrite action readIndex outIndex finalTransform = 
    goto readIndex ++ ", [ " ++ action ++ goto readIndex ++ "," ++ "]" ++ 
    goto outIndex ++ finalTransform ++ "."

In [27]:
double addr ss1 ss2 = copyTo addr ss1 ss2 ++ addTo ss1 addr ss2

In [28]:
fromAscii x = goto x ++ replicate 48 '-'
toAscii x = goto x ++ replicate 48 '+'

In [29]:
home = 0
inputCell = 1
outputCell = 2
safeSpace1 = 3
safeSpace2 = 4

## The Final Program:

In [35]:
"-" ++ readDoWrite (
        double outputCell safeSpace1 safeSpace2 ++
        fromAscii inputCell ++ 
        addTo inputCell outputCell safeSpace1
    ) 1 2 (toAscii outputCell)

"-+ [ - < + ] ->, [ + [ - < + ] ->>>[-]+ [ - < + ] ->>>>[-]+ [ - < + ] ->>[-+ [ - < + ] ->>>++ [ - < + ] ->>>>++ [ - < + ] ->>]+ [ - < + ] ->>[-]+ [ - < + ] ->>>>[-+ [ - < + ] ->>++ [ - < + ] ->>>>]+ [ - < + ] ->>>>[-]+ [ - < + ] ->>>[-+ [ - < + ] ->>++ [ - < + ] ->>>>++ [ - < + ] ->>>]+ [ - < + ] ->>>[-]+ [ - < + ] ->>>>[-+ [ - < + ] ->>>++ [ - < + ] ->>>>]+ [ - < + ] ->------------------------------------------------+ [ - < + ] ->>>[-]+ [ - < + ] ->[-+ [ - < + ] ->>++ [ - < + ] ->>>++ [ - < + ] ->]+ [ - < + ] ->[-]+ [ - < + ] ->>>[-+ [ - < + ] ->++ [ - < + ] ->>>]+ [ - < + ] ->,]+ [ - < + ] ->>+ [ - < + ] ->>++++++++++++++++++++++++++++++++++++++++++++++++."